In [0]:
%sql
--describe table orders;
--describe table customers;
describe table order_details;

In [0]:
%sql
-- Find customers who placed orders above the average order amount.

SELECT c.first_name,c.last_name,o.total_amount
FROM customers c INNER JOIN orders o ON c.customer_id = o.customer_id WHERE o.total_amount >
(
SELECT avg(total_amount) FROM orders
)

In [0]:
%sql
-- List all products that have been sold more than 2 times in total.
SELECT od.product_name AS PRODUCT_NAME,sum(od.quantity) AS TOTAL_SOLD 
FROM order_details od GROUP BY od.product_name HAVING sum(od.quantity) > 
(SELECT 2)

CTE's Common Table Expressions


In [0]:
%sql
--  List customers and their total quantity of products ordered.
WITH customer_details AS (
  SELECT c.customer_id,c.first_name, sum(od.quantity) AS Total_Quantity
  FROM customers c 
  INNER JOIN orders o ON c.customer_id = o.customer_id
  INNER JOIN order_details od ON od.order_id = o.order_id
  GROUP BY c.customer_id,c.first_name
)

SELECT * FROM customer_details WHERE Total_Quantity > 2 

In [0]:
%sql
--Find products with their total sales revenue (price * quantity).
WITH Product_Sales AS (
SELECT od.product_name,SUM (od.price_each * od.quantity) AS Total_Revenue
FROM order_details od GROUP BY od.product_name
)

SELECT * FROM Product_Sales where Total_Revenue > 10000;

CASE WHEN

In [0]:
%sql
--Categorize each order as 'Small', 'Medium', 'Large'.
SELECT o.order_id,o.total_amount,
CASE
    WHEN o.total_amount < 1000 THEN 'Small'
    WHEN o.total_amount BETWEEN 1000 and 2500 THEN 'Medium'
    ELSE 'Large'
END AS ORDER_SIZE
FROM orders o;    

In [0]:
%sql
-- Categorize each order as 'Small', 'Medium', 'Large'.

WITH categorized_orders AS (
    SELECT o.order_id, o.total_amount, 
          CASE
              WHEN o.total_amount < 1000 THEN 'SMALL'
              WHEN o.total_amount BETWEEN 1000 AND 1500 THEN 'MEDIUM'
              ELSE 'LARGE'
          END AS ORDER_SIZE 
    FROM orders o
)
SELECT * 
FROM categorized_orders 
WHERE ORDER_SIZE = 'LARGE'

In [0]:
%sql
--Label customers as 'New' or 'Returning' based on order count.
SELECT c.first_name,count(o.order_id) AS total_orders,
CASE
    WHEN count(o.order_id) = 1 THEN 'New'
    WHEN count(o.order_id) > 1 THEN 'Returning'
    ELSE 'No Orders'
END AS Customer_Type
FROM customers c 
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name

FUNCTIONS (STRING+DATE)

In [0]:
%sql
SELECT UPPER(CONCAT(c.first_name," ",c.last_name)) AS FULL_NAME from customers c 

In [0]:
%sql
SELECT LOWER(c.first_name) AS FIRST_NAME_UPPERCASE from customers c 

In [0]:
%sql
SELECT UPPER(CONCAT(c.first_name," ",c.last_name)) AS FULL_NAME from customers c 

In [0]:
%sql
-- Extract month of each order & group orders by month.
SELECT MONTH(o.order_date) AS order_month, count(o.order_id) AS _total_orders 
FROM orders o 
GROUP BY 1
ORDER BY 1

In [0]:
%sql
-- List each customer's name + their total spend + spending category.
WITH Customer_Spend AS(
  SELECT c.customer_id,CONCAT(c.first_name," ",c.last_name) AS Full_Name,
  sum(od.price_each * od.quantity) AS Total_Spent
  FROM customers c
  INNER JOIN orders o ON c.customer_id = o.customer_id
  INNER JOIN order_details od ON od.order_id = o.order_id
  GROUP BY c.customer_id,c.first_name,c.last_name
)

SELECT Full_Name , Total_Spent, 
       CASE
           WHEN total_spent > 10000 THEN 'Gold'
           WHEN total_spent BETWEEN 2000 AND 10000 THEN 'Silver'
           ELSE 'Bronze'
        END AS Customer_Tier
FROM Customer_Spend;